In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfuly.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InterectiveBrowserCredential in case DefaultAzureCrendential
    credential = InteractiveBrowserCredential()

ModuleNotFoundError: No module named 'azure'

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

NameError: name 'MLClient' is not defined

In [ ]:
%%writefile Cosumo/sorvetes-train.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the sorvets dateset
print("Loading Data...")
sorvetes = pd.read_csv("sorvetes.csv")

# serarate features and labels
X, y = sorvetes[["Temperatura"]].values, sorvetes["Ventes"].values

# separate features and labels
X_train, X_test, y_train, y_test = traint_test_split(X, y, test_size=0.20, random_state=0)

# set regularization regression hyperparameter
reg = 0.01

# train logistic regression model

print("Trainning a logistic regression model with regularization rate of", reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print("Acuracy:", acc)

# calculate AUC

y_score = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_score[:,1])
print("AUC: " + str(auc))


In [ ]:
from azure.ai.ml import comand
# configure job
job = comand(
    code="./Comsuno",
    comand = "python sorvetes-training.py",
    environment="AzureML-sklearning-0.24-ubuntu18.04-py37-cpu@lastest",
    compute = "aml-cluter",
    display_name="sorvetes-pythonv2-tarin",
    experiment_name="sorvetes-training"
)

#submit job

returned_job = ml_client.creat_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)